# Phase 1 - Data Collection and Data Preprocessing

In [558]:
# from idlelib.editor import darwin
# !pip install pandas

# Data Loading

In [559]:
import pandas as pd
from nltk import bigrams

file_path = 'urdu-sentiment-corpus-v1.tsv'
data = pd.read_csv(file_path, sep='\t')
data.head()


,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P


# Loading Stop Words

In [560]:
stopWords=[]
with open('urdu_stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f:
        stopWords.append(line.strip()) # used strip to remove leading and trailing white spaces
stopWords


['آئی',
 'آئے',
 'آج',
 'آخر',
 'آخرکبر',
 'آدهی',
 'آًب',
 'آٹھ',
 'آیب',
 'اة',
 'اخبزت',
 'اختتبم',
 'ادھر',
 'ارد',
 'اردگرد',
 'ارکبى',
 'اش',
 'اضتعوبل',
 'اضتعوبلات',
 'اضطرذ',
 'اضکب',
 'اضکی',
 'اضکے',
 'اطراف',
 'اغیب',
 'افراد',
 'الگ',
 'اور',
 'اوًچب',
 'اوًچبئی',
 'اوًچی',
 'اوًچے',
 'اى',
 'اً',
 'اًذر',
 'اًہیں',
 'اٹھبًب',
 'اپٌب',
 'اپٌے',
 'اچھب',
 'اچھی',
 'اچھے',
 'اکثر',
 'اکٹھب',
 'اکٹھی',
 'اکٹھے',
 'اکیلا',
 'اکیلی',
 'اکیلے',
 'اگرچہ',
 'اہن',
 'ایطے',
 'ایک',
 'ب',
 'ت',
 'تبزٍ',
 'تت',
 'تر',
 'ترتیت',
 'تریي',
 'تعذاد',
 'تن',
 'تو',
 'توبم',
 'توہی',
 'توہیں',
 'تٌہب',
 'تک',
 'تھب',
 'تھوڑا',
 'تھوڑی',
 'تھوڑے',
 'تھی',
 'تھے',
 'تیي',
 'ثب',
 'ثبئیں',
 'ثبترتیت',
 'ثبری',
 'ثبرے',
 'ثبعث',
 'ثبلا',
 'ثبلترتیت',
 'ثبہر',
 'ثدبئے',
 'ثرآں',
 'ثراں',
 'ثرش',
 'ثعذ',
 'ثغیر',
 'ثلٌذ',
 'ثلٌذوثبلا',
 'ثلکہ',
 'ثي',
 'ثٌب',
 'ثٌبرہب',
 'ثٌبرہی',
 'ثٌبرہے',
 'ثٌبًب',
 'ثٌذ',
 'ثٌذکرو',
 'ثٌذکرًب',
 'ثٌذی',
 'ثڑا',
 'ثڑوں',
 'ثڑی',
 'ثڑے',
 'ثھر',
 'ثھرا',
 'ثھر

# Removing Stop Words

In [561]:
def remove_stop_words(text):
    text = text.split()
    filtered_text = []
    for word in text:
        if word not in stopWords:
            filtered_text.append(word)
    return ' '.join(filtered_text)
after_stop_wors=remove_stop_words('میں نے ایک  کتاب پڑھی ہے')  # Demo removing stop words
print(after_stop_wors)

میں کتاب پڑھی


In [562]:
data['Tweet']=data['Tweet'].apply(remove_stop_words)
data.head() 

,Tweet,Class
0,میں ایٹم بم بنایا ۔۔۔۔او بھائی ایٹم بمب کوٹ لک...,P
1,چندے سے انقلاب عمران خان وزیر اعظم نہیں بن سکتے,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل نائب صدر فضا میں ، 130,000 فٹ بل...",P
4,ابھی اسکی لہریں کبھی کبھی آ جاتی یار :أْ,P


# Removing URL

In [563]:
import re
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')   
    return pattern.sub(r'', text)

after_url=remove_url('میں نے ایک  کتاب پڑھی ہے https://www.google.com')  # Demo removing URL
after_url
    

'میں نے ایک  کتاب پڑھی ہے '

In [564]:
data["Tweet"]=data["Tweet"].apply(remove_url)
data.head()

,Tweet,Class
0,میں ایٹم بم بنایا ۔۔۔۔او بھائی ایٹم بمب کوٹ لک...,P
1,چندے سے انقلاب عمران خان وزیر اعظم نہیں بن سکتے,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل نائب صدر فضا میں ، 130,000 فٹ بل...",P
4,ابھی اسکی لہریں کبھی کبھی آ جاتی یار :أْ,P


# Removing Punctuations

In [565]:
import string
# got these punctuations from Chatgpt and appended english one's also as some people use those as punc
urdu_punctuation = {
    "۔",  # Full stop (وقفہ)
    "،",  # Comma (وقفہ)
    "؛",  # Semicolon (نیم وقفہ)
    ":",  # Colon (دو نقطے)
    "؟",  # Question mark (سوالیہ نشان)
    "?",  # Question mark (سوالیہ نشان)
    "!",  # Exclamation mark (ندائیہ نشان)
    "\"\"",  # Quotation marks (اقتباس)
    "''",  # Single quotes (ایک قوسین)
    "()",  # Parentheses (قوسین)
    "-",  # Hyphen (ربط نشان)
    "…",  # Ellipsis (تین نقطے)
    "/",  # Slash (سلیش)
    "@",  # At symbol (ایٹ)
    "#",  # Hash (ہیش)
    "%",  # Percentage sign (فیصد)
}

# Add unique English punctuations
urdu_punctuation.update(string.punctuation)

new_urdu_punc = ''.join(urdu_punctuation)
print(new_urdu_punc)


`+~؟?؛…^""#:)<۔},{.-|[>_]'*''\/!%،=$()@&"(;


In [566]:
def remove_punc(text):
    for word in text:
        if word in new_urdu_punc:
            text = text.replace(word, "")
    return text

after_punc=remove_punc('میں نے- ایک  کتاب? پڑھی>< ہے!')  # Demo removing Punctuation
print(after_punc)

میں نے ایک  کتاب پڑھی ہے


In [567]:
data["Tweet"]=data["Tweet"].apply(remove_punc)
data.head()

,Tweet,Class
0,میں ایٹم بم بنایا او بھائی ایٹم بمب کوٹ لکھپت ...,P
1,چندے سے انقلاب عمران خان وزیر اعظم نہیں بن سکتے,N
2,ٹویٹر کا خیال کیسے آیا,O
3,سرچ انجن گوگل نائب صدر فضا میں 130000 فٹ بلند...,P
4,ابھی اسکی لہریں کبھی کبھی آ جاتی یار أْ,P


# Removing Emojis


# Removing hashtags

In [568]:
def remove_hashtags(text):
    return re.sub(r'#\w+', '', text)



# Example usage
test_hashtags = pd.DataFrame({
    'Tweet': [
        'یہ ایک #کتاب ہے',  # This is a #book
        'آج موسم #خوشگوار ہے',  # The weather is #pleasant today
        'مجھے #چائے پسند ہے'  # I like #tea
    ],
    'Class': ['P', 'N', 'O']
})

# Apply the function to the 'Tweet' column
test_hashtags['Tweet'] = test_hashtags['Tweet'].apply(remove_hashtags)
print(test_hashtags)

           Tweet Class
0     یہ ایک  ہے     P
1    آج موسم  ہے     N
2  مجھے  پسند ہے     O


In [569]:
data["Tweet"]=data["Tweet"].apply(remove_hashtags)
data.head()

,Tweet,Class
0,میں ایٹم بم بنایا او بھائی ایٹم بمب کوٹ لکھپت ...,P
1,چندے سے انقلاب عمران خان وزیر اعظم نہیں بن سکتے,N
2,ٹویٹر کا خیال کیسے آیا,O
3,سرچ انجن گوگل نائب صدر فضا میں 130000 فٹ بلند...,P
4,ابھی اسکی لہریں کبھی کبھی آ جاتی یار أْ,P


# Removing short Convo

In [570]:
def remove_short_convo(text):
    # Drop rows where the 'Tweet' column has less than 3 words
    return text[text['Tweet'].apply(lambda x: len(x.split()) >= 3)]


# text['Tweet'].apply(lambda x: len(x.split()) >= 3) applies a lambda function to each element in the 'Tweet' column.
# The lambda function returns True if the tweet contains 3 or more words, and False otherwise.
# This results in a Series of boolean values.
# Apply the Boolean Mask:  
# text[text['Tweet'].apply(lambda x: len(x.split()) >= 3)] uses the boolean mask to filter the DataFrame.
# Only the rows where the mask is True are selected


test_list = pd.DataFrame({
    'Tweet': [
        'کتاب ہے',  # This is a book
        'آج موسم خوشگوار ہے',  # The weather is pleasant today
        'مجھے چائے پسند ہے'  # I like tea
    ],
    'Class': ['P', 'N', 'O']
})

test_list=remove_short_convo(test_list)
print(test_list)

                Tweet Class
1  آج موسم خوشگوار ہے     N
2   مجھے چائے پسند ہے     O


In [571]:
data=remove_short_convo(data)
data.head()

,Tweet,Class
0,میں ایٹم بم بنایا او بھائی ایٹم بمب کوٹ لکھپت ...,P
1,چندے سے انقلاب عمران خان وزیر اعظم نہیں بن سکتے,N
2,ٹویٹر کا خیال کیسے آیا,O
3,سرچ انجن گوگل نائب صدر فضا میں 130000 فٹ بلند...,P
4,ابھی اسکی لہریں کبھی کبھی آ جاتی یار أْ,P


# Phase 2 - Stemming & Lemmatization

# Stemming


In [572]:
def urdu_stemmer(word):
    # List of common prefixes and suffixes in Urdu
    prefixes = ['بے', 'نا', 'غیر', 'پرا', 'و', 'ائين', 'تا', 'يان', 'ناك', 'ذو', 'بد','ہم']  # Add more prefixes as needed
    suffixes = ['وں', 'یں', 'ا', 'ے', 'ی', 'وا','و', 'ائين', 'تا', 'يان', 'ناك', 'ذو', 'بد' ,'وں']  # Add more suffixes as needed

    # Remove prefixes
    for prefix in prefixes:
        if word.startswith(prefix):
            word = word[len(prefix):]
            break  # Only remove the first matching prefix

    # Remove suffixes
    for suffix in suffixes:
        if word.endswith(suffix):
            word = word[:-len(suffix)]
            break  # Only remove the first matching suffix

    return word

# Example text
text = """
 کتابوں غیرمعمولی ناانصافی بےکار خوشیوں غمگین ناگوار 
 پڑھائی خوشبو پراگندہ بےروزگار بےوقوف کتابیں انسانوں 
ہمراہ بےمعنی بچوں مصیبتوں غیرضروری خوشحال ہمسفر کامیابیاں 
غیرذمہ دار پریشانیاں زندگی مسائل کامیاب سوالات
"""
tokens = text.split()
stemmed_tokens = [urdu_stemmer(token) for token in tokens]

print(stemmed_tokens)  # Output: ['کتاب']


['کتاب', 'معمول', 'انصاف', 'کار', 'خوشی', 'غمگین', 'گوار', 'پڑھائ', 'خوشب', 'گندہ', 'روزگار', 'وقوف', 'کتاب', 'انسان', 'راہ', 'معن', 'بچ', 'مصیبت', 'ضرور', 'خوشحال', 'سفر', 'کامیابیاں', 'ذمہ', 'دار', 'پریشانیاں', 'زندگ', 'مسائل', 'کامیاب', 'سوالات']


In [573]:
data_stem=data.copy()
data_stem['Tweet']=data_stem['Tweet'].apply(lambda x: ' '.join([urdu_stemmer(word) for word in x.split()]))
data_stem.head()

,Tweet,Class
0,م ایٹم بم بنای ا بھائ ایٹم بمب کوٹ لکھپت ال ات...,P
1,چند س انقلاب عمران خان زیر اعظم نہ بن سکت,N
2,ٹویٹر ک خیال کیس آی,O
3,سرچ انجن گوگل ئب صدر فض م 130000 فٹ بلند چھلان...,P
4,ابھ اسک لہر کبھ کبھ آ جات یار أْ,P


# Lemmatization

In [574]:
# Custom lemmatizer example
lemmatization_dict = {
    'کتابیں': 'کتاب',
    'لڑکیاں': 'لڑکی',
    # Add more word mappings
}

def lemmatize(word):
    return lemmatization_dict.get(word, word)

# Example usage
sample_text = "کتابیں لڑکیاں"
lemmatized_sample = ' '.join([lemmatize(word) for word in sample_text.split()])
print(lemmatized_sample)

کتاب لڑکی


# Custom dictionary-based lemmatization

In [575]:
import lemma_dict_file as ldf

def lemmatize_urdu(tweets):
    lemmatized_text = ' '.join(ldf.urdu_lemma_dict.get(word,word) for word in tweets.split())
    return lemmatized_text

data_lemmatized = data.copy()
data_lemmatized['Tweet'] = data_lemmatized['Tweet'].apply(lemmatize_urdu)
data_lemmatized.head()




,Tweet,Class
0,میں ایٹم بم بنا او بھائی ایٹم بمب کوٹ لکھپت وا...,P
1,چندے سے انقلاب عمران خان وزیر اعظم نہیں بن سکتے,N
2,ٹویٹر کا خیال کیسے آ,O
3,سرچ انجن گوگل نائب صدر فضا میں 130000 فٹ بلندی...,P
4,ابھی اسکی لہریں کبھی کبھی آ جا یار أْ,P


# Phase 3 - Feature Extraction

# Sample word tokenization

In [576]:
import  nltk
nltk.download('punkt') # for not straight forward languages uses a model to tokenize
from nltk.tokenize import word_tokenize

text = "یہ ایک مثال جملہ ہے۔"

tokenized=word_tokenize(text)
print(tokenized)



['یہ', 'ایک', 'مثال', 'جملہ', 'ہے۔']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [577]:
# applying on lemmatized
data_tokenized = data_lemmatized.copy()

data_tokenized['Tweet'] = data_tokenized['Tweet'].apply(word_tokenize)
data_tokenized.head()

,Tweet,Class
0,"[میں, ایٹم, بم, بنا, او, بھائی, ایٹم, بمب, کوٹ...",P
1,"[چندے, سے, انقلاب, عمران, خان, وزیر, اعظم, نہی...",N
2,"[ٹویٹر, کا, خیال, کیسے, آ]",O
3,"[سرچ, انجن, گوگل, نائب, صدر, فضا, میں, 130000,...",P
4,"[ابھی, اسکی, لہریں, کبھی, کبھی, آ, جا, یار, أْ]",P


# TF-IDF Vectorization



In [578]:
# !pip install scikit-learn

In [579]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_lemmatized['Tweet'])

feature_names=tfidf_vectorizer.get_feature_names_out()
tfidf_scores=tfidf_matrix.toarray().sum(axis=0)
tfidf_df=pd.DataFrame({'Word':feature_names,'TF-IDF':tfidf_scores})
tfidf_df=tfidf_df.sort_values('TF-IDF',ascending=False)
tfidf_df.head(10)


,Word,TF-IDF
3097,میں,37.987015
2130,سے,32.446385
3847,کا,30.213424
3916,کر,25.049767
4031,کو,24.150365
3243,نہیں,18.251650
909,بھی,17.056891
1138,جا,15.184444
4498,یہ,13.848613
3367,وہ,13.799112


# Word2Vec Embeddings


In [580]:
!pip install gensim
# !pip install --upgrade numpy

DEPRECATION: Loading egg at c:\users\abdul\appdata\local\programs\python\python312\lib\site-packages\mlrose-1.3.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [581]:
from gensim.models import Word2Vec

model=Word2Vec(data_tokenized['Tweet'],min_count=1,window=5,sg=0,vector_size=10)
similar_words = model.wv.most_similar("اچھا", topn=5)
similar_df=pd.DataFrame(similar_words,columns=['Word','Similarity'])
similar_df


,Word,Similarity
0,جانے,0.920974
1,فیروز,0.857997
2,دقیق,0.846315
3,سوال,0.839393
4,بھوک,0.837931


# Phase 4 - N-Gram Analysis



In [582]:
import  nltk
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

analysis_data=data.copy()

analysis_data["Tweet"]=analysis_data["Tweet"].apply(nltk.word_tokenize)

unigrams=[word for tokens in analysis_data["Tweet"] for word in tokens]
bigrams=[bigram for tokens in analysis_data["Tweet"] for bigram in ngrams(tokens,2)]
trigrams=[trigram for tokens in analysis_data["Tweet"] for trigram in ngrams(tokens,3)]

unigram_freq=Counter(unigrams)
bigram_freq=Counter(bigrams)
trigram_freq=Counter(trigrams)

top_10_bigrams =bigram_freq.most_common(10)
top_10_trigrams =trigram_freq.most_common(10)

print("Top 10 Bigrams:")
for bigram, freq in top_10_bigrams:
    print(f"{bigram}: {freq}")

print("\nTop 10 Trigrams:")
for trigram, freq in top_10_trigrams:
    print(f"{trigram}: {freq}")

Top 10 Bigrams:
('سے', 'انقلاب'): 26
('چندے', 'سے'): 25
('عمران', 'خان'): 21
('طاہر', 'القادری'): 19
('اسلام', 'آباد'): 17
('تحریک', 'انصاف'): 14
('عوامی', 'تحریک'): 13
('سیدنا', 'عمر'): 12
('وزیر', 'اعظم'): 11
('کا', 'اعلان'): 11

Top 10 Trigrams:
('چندے', 'سے', 'انقلاب'): 23
('ایم', 'کیو', 'ایم'): 10
('سیدنا', 'عمر', 'فاروق'): 8
('ڈاکٹر', 'طاہر', 'القادری'): 6
('نیا', 'اسلامی', 'سال'): 5
('اسلامی', 'سال', 'مبارک'): 5
('اسلام', 'آباد', 'میں'): 5
('ملک', 'بھر', 'میں'): 5
('عمر', 'فاروق', 'رضی'): 5
('فاروق', 'رضی', 'اللہ'): 5


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Phase 5 - Sentiment Analysis

In [583]:
from sklearn.model_selection import train_test_split
print(data.isnull().sum())
data = data.dropna()
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['Tweet'], data['Class'], test_size=0.20, random_state=90)



# Assuming tfidf_matrix is already created
X_train_tfidf = tfidf_matrix[:len(X_train)] #  This line takes the first len(X_train) rows from tfidf_matrix and assigns them to X_train_tfidf. These rows correspond to the training set features.
X_test_tfidf = tfidf_matrix[len(X_train):len(X_train) + len(X_test)] # This line takes the remaining rows from tfidf_matrix (starting from len(X_train) to the end) and assigns them to X_test_tfidf. These rows correspond to the test set features.





Tweet    0
Class    1
dtype: int64


# Applying Logistic Regression

In [584]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model=LogisticRegression()
model.fit(X_train_tfidf,y_train)
y_pred=model.predict(X_test_tfidf)


# Evaluating the Model

In [585]:
# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted',zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Accuracy: 0.53
Precision: 0.54
Recall: 0.53
F1-score: 0.51
